## Задание 1

В файле содержится информация о покупках людей

- id – означает покупку (в одну покупку входят все товары, купленные пользователем во время 1 похода в магазин)
- Товар – наименование товара
- Количество – число единиц купленного товара

Воспользуйтесь этими данными и выясните, какие пары товаров пользователи чаще всего покупают вместе. По сути, вам необходимо найти паттерны покупок, что позволит оптимизировать размещение продуктов в магазине, для удобства пользователей и увеличения выручки.

Напишите код на python для получения нужной таблицы и укажите 5 наиболее распространённых паттернов.

## Шаг 1: Подготовка данных

Импортируем необходимые библиотеки:

In [23]:
import pandas as pd
import numpy as np
from itertools import *

Загрузим датасет и взглянем на него:

In [24]:
url = 'test1_completed.csv'
df = pd.read_csv(url)

In [25]:
df.head(5)

,id,Товар,Количество
0,17119,Лимон,1.1
1,17119,Лимон оранжевый,0.7
2,17119,Лук-порей,10.0
3,17119,Лук репчатый,2.5
4,17119,Малина свежая,1.0


Подготовим датасет к обработке:
- Переназовем колонку
- Уберем количество(оно нам не понадобится для решения задачи)

In [26]:
df.rename(columns={'Товар':'goods'}, inplace=True)
df.drop(columns='Количество', inplace=True)

In [27]:
df.head(5)

,id,goods
0,17119,Лимон
1,17119,Лимон оранжевый
2,17119,Лук-порей
3,17119,Лук репчатый
4,17119,Малина свежая


## Шаг 3: Анализ данных

Требуется найти пары товаров, а затем посчитать их встречаемость.

Посмотрим сколько всего у нас различных товаров:

In [28]:
df['goods'].nunique()

199

Для данного количества товаров, возможно 19701 сочетания без повторения (вычислим по формуле 199! / 2! * (199-2)!), проверим себя, больше пар получиться не должно

## Шаг 3: Обработка данных

Алгоритм решения задачи следующий:
1. Возьмем каждый заказ и просчитаем все возможные сочетания товаров в нем
2. Будем записывать в словарь все новые пары товаров и увеличивать счетчик на 1
3. Если попадается пара которое уже есть в словаре, увеличиваем счетчик на 1, не добавляя в словарь пару
4. Так как товары идут в произвольном порядке проверим и учтем, одинаковые пары в разном порядке

In [29]:
pair_goods = {}

for ids in df.id.unique():                                  # Получим список уникальных id и проитерируемся по каждому
    for i in permutations(df.query('id == @ids').goods, 2): # Для каждого id составим все комбинации товара
        if i not in pair_goods:
            tmp_pair_revert = i[1],i[0]
            if tmp_pair_revert not in pair_goods:           # Составим словарь пар и посчитаем их
                pair_goods[i] = 1
        else:
            pair_goods[i] = pair_goods[i]+1

Проверим длинну словаря:

In [30]:
len(pair_goods)

19697

Все хорошо, пар, не больше максимально возможного

Загрузим все в датафрейм:

In [31]:
df_pair_goods = pd.DataFrame(pair_goods, index=[0]).T.reset_index()
df_pair_goods.rename(columns={'level_0':'1_Товар', 'level_1':'2_Товар', 0:'Встречаемость'}, inplace=True)
df_pair_goods.head(5)

,1_Товар,2_Товар,Встречаемость
0,Лимон,Лимон оранжевый,31
1,Лимон,Лук-порей,40
2,Лимон,Лук репчатый,114
3,Лимон,Малина свежая,25
4,Лимон,Морковь немытая,80


Наконец выведем топ 5 пар, по встречаемости в покупке

In [32]:
df_pair_goods.sort_values(by=['Встречаемость'],ascending=False).head(5)

,1_Товар,2_Товар,Встречаемость
291,Огурцы Луховицкие,Укроп,431
297,Петрушка,Укроп,408
3630,Арбуз,Огурцы Луховицкие,345
855,Кабачки,Огурцы Луховицкие,326
443,Кинза,Укроп,303
